In [1]:
import re

class PnaToSpiceConverter:
    # Updated layer mapping
    LAYER_MAPPING = {
        84: "metal0",
        85: "via0",
        11: "metal1",
        12: "via1",
        13: "metal2",
        14: "via2",
        15: "metal3",
        16: "via3",
        17: "metal4",
        18: "via4",
        19: "metal5",
        20: "via5",
        21: "metal6",
        22: "via6",
        23: "metal7",
        24: "via7",
        25: "metal8",
        26: "via8",
        27: "metal9",
        91: "via9",
        92: "metal10",
        93: "via10",
        94: "metal11",
        95: "via11",
        96: "metal12",
    }

    def __init__(self, vss_file, vdd_file, output_file):
        self.vss_file = vss_file
        self.vdd_file = vdd_file
        self.output_file = output_file
        self.parsed_data = []
        self.resistor_id = 0

    def parse_pna_file(self, file_path):
        """Parse a PNA file and extract relevant fields."""
        parsed_entries = []
        try:
            with open(file_path, 'r') as file:
                lines = file.readlines()
                for line in lines:
                    if not line.startswith("*"):  # Skip comments and metadata
                        match = re.match(
                            r"(\d+)\s+(\w+)\s+([\d.]+)\s+([\d.]+)\s+([\d.]+)\s+(\w+)\s+(\d+)\s+\(([\d.]+)\s+([\d.]+)\s+([\d.]+)\s+([\d.]+)\)",
                            line.strip()
                        )
                        if match:
                            groups = match.groups()
                            parsed_entries.append({
                                "id": int(groups[0]),
                                "type": groups[1],
                                "resistance": float(groups[2]),
                                "vdrop": float(groups[3]),
                                "current": float(groups[4]),
                                "direction": groups[5],
                                "layer": int(groups[6]),
                                "xlo": float(groups[7]),
                                "ylo": float(groups[8]),
                                "xhi": float(groups[9]),
                                "yhi": float(groups[10]),
                            })
        except IOError as e:
            print(f"Error reading file {file_path}: {e}")
        return parsed_entries

    def convert_to_spice(self):
        """Convert parsed PNA data into SPICE netlist format."""
        spice_netlist = []
        for entry in self.parsed_data:
            layer_name = self.LAYER_MAPPING.get(entry["layer"], f"unknown_layer_{entry['layer']}")
            
            node_start = f"n1_{layer_name}_{entry['xlo']:.3f}_{entry['ylo']:.3f}"
            node_end = f"n1_{layer_name}_{entry['xhi']:.3f}_{entry['yhi']:.3f}"
            
            spice_entry = f"R{self.resistor_id} {node_start} {node_end} {entry['resistance']:.6f}"
            spice_netlist.append(spice_entry)
            
            self.resistor_id += 1
        return spice_netlist

    def save_spice_file(self, spice_netlist):
        """Save the SPICE netlist to an output file."""
        try:
            with open(self.output_file, 'w') as file:
                file.write("* SPICE netlist generated from PNA files\n")
                file.write(f"* VSS file: {self.vss_file}\n")
                file.write(f"* VDD file: {self.vdd_file}\n")
                file.write("\n".join(spice_netlist))
            print(f"SPICE netlist saved to {self.output_file}")
        except IOError as e:
            print(f"Error writing to file {self.output_file}: {e}")

    def process_files(self):
        """Main processing pipeline to parse and convert files."""
        self.parsed_data.extend(self.parse_pna_file(self.vss_file))
        self.parsed_data.extend(self.parse_pna_file(self.vdd_file))
        
        spice_netlist = self.convert_to_spice()
        
        self.save_spice_file(spice_netlist)

# Example usage
vss_path = "example/aes_cipher_top.VSS.icc2.pna"
vdd_path = "example/aes_cipher_top.VDD.icc2.pna"
output_path = "example/netlist_resistance.sp"

converter = PnaToSpiceConverter(vss_path, vdd_path, output_path)
converter.process_files()

SPICE netlist saved to example/netlist_resistance.sp
